In [ ]:
import openai
import argparse
import os
import pyterrier as pt
import pandas as pd
from utils import *
from pyterrier.measures import *
from pandarallel import pandarallel
from tqdm import tqdm_notebook as tqdm

In [ ]:
# 导入模块
import os
# 设置代理
proxy = ''
os.environ['http_proxy'] = proxy
os.environ['HTTP_PROXY'] = proxy
os.environ['https_proxy'] = proxy
os.environ['HTTPS_PROXY'] = proxy

In [ ]:
# hyperparameters
dataset_name = 'irds:beir/climate-fever' # irds:beir/scifact/test irds:beir/fiqa/test
doc_field = 'text'
query_field = 'text'
N = 5 # LLM candidates
K = 5 # PRF candidates
n = 3
k = 3

In [ ]:
os.environ["JAVA_HOME"]="your java path"
import pyterrier as pt
pt.init(version=5.7, helper_version="0.0.7")

In [ ]:
dataset = pt.get_dataset(dataset_name)
indexref = './index/{}'.format(dataset_name.replace('/', '-'))
index = pt.IndexFactory.of(indexref)

In [ ]:
rewrite_retriever = pt.BatchRetrieve(index, wmodel='BM25', metadata=['docno',doc_field], num_results=K)
test_retriever = pt.BatchRetrieve(index, wmodel='BM25', metadata=['docno',doc_field])
eval_metrics = [nDCG@1, nDCG@5, nDCG@10, nDCG@100, nDCG@1000,
                            AP@1, AP@5, AP@10, AP@100, AP@1000,
                            R@1, R@5, R@10, R@100, R@1000,
                            P@1, P@5, P@10, P@100, P@1000,
                            RR@1, RR@5, RR@10, RR@100, RR@1000]

In [ ]:
dataset.get_topics()

In [ ]:
df = dataset.get_topics()
length = 0
for i in range(df.shape[0]):
    length += len(df.loc[i,'query'])
length /= df.shape[0]
length

In [ ]:
df = pd.read_csv('rewrites/irds:beir-climate-fever-qqdzs-N3-K3-n3-k3-rewrite.csv')
df

In [ ]:
retrieve_input = dataset.get_topics(query_field)
retrieved_results = rewrite_retriever(retrieve_input)

In [ ]:
for i in range(df.shape[0]):
    query = df.loc[i,'query_0']
    doc_lis = retrieved_results[retrieved_results['query']==query][doc_field].tolist()
    doc_lis = np.array([x[:1024] for x in doc_lis])
    df.loc[i,'query'] = df.loc[i,'query'] + ''.join(doc_lis[:3])

In [ ]:
df['qid'] = df['qid'].astype(str).astype(object)

In [ ]:
results = pt.Experiment(
            [pt.rewrite.tokenise() >> test_retriever.parallel(9)],
            df[['qid','query']],
            dataset.get_qrels(),
            eval_metrics = eval_metrics
        )

In [ ]:
results.to_csv('results/irds:beir-climate-fever-qqdzs-concatprf-N3-K3-n3-k3.csv')